# This is an implemetation of a Preceptron

The code down below is part of the *lib.py* file, which will contain the:
* random weights generation
* the prediction function *threshold function*
* the weighted sum function
* and the training function

In [171]:
import numpy as np

In [172]:
def random_weights(X, random_state: int):
    rand = np.random.RandomState(random_state)
    w = rand.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
    return w

In [173]:
def predict(X, w):
    return np.where(net_input(X, w) >= 0.0, 1, -1)

In [174]:
def net_input(X, w):
    return np.dot(X, w[1:]) + w[0]

In [175]:
def fit(X, y, eta=0.001, n_iter=100):
    errors = []
    w = random_weights(X, random_state=1)
    for exemplar in range(n_iter):
        error = 0
        for xi, target in zip(X, y):
            delta = eta * (target - predict(xi, w))
            w[1:] += delta * xi
            w[0] += delta
            error += int(delta != 0.0)
        errors.append(error)
    return w, errors

In [176]:
import numpy as np
import pandas as pd
import altair as alt

In [177]:
def species_generator(mu1, sigma1, mu2, sigma2, n_samples, target, seed):
    # mu1 is the mean weight
    # mu2 is the  mean wingspan
    rand = np.random.RandomState(seed)
    f1 = rand.normal(mu1, sigma1, n_samples)
    f2 = rand.normal(mu2, sigma2, n_samples)
    X = np.array([f1, f2])
    X = X.transpose()
    y = np.full((n_samples), target)
    return X, y

In [178]:
albatross_weight_mean = 9000 # in grams
albatross_weight_variance =  800 # in grams
albatross_wingspan_mean = 300 # in cm
albatross_wingspan_variance = 20 # in cm 
n_samples = 100
target = 1
seed = 100

# aX: feature matrix (weight, wingspan)
# ay: target value (1)
aX, ay = species_generator(albatross_weight_mean, albatross_weight_variance,
                           albatross_wingspan_mean, albatross_wingspan_variance,
                           n_samples,target,seed )


albatross_dic = {'weight-(gm)': aX[:,0],
                 'wingspan-(cm)': aX[:,1], 
                 'species': ay,
                 'url': "https://raw.githubusercontent.com/pabloinsente/nn-mod-cog/master/notebooks/images/albatross.png"}


# put values in a relational table (pandas dataframe)
albatross_df = pd.DataFrame(albatross_dic)

In [179]:
owl_weight_mean = 1000 # in grams
owl_weight_variance =  200 # in grams
owl_wingspan_mean = 100 # in cm
owl_wingspan_variance = 15 # in cm
n_samples = 100
target = -1
seed = 100

# oX: feature matrix (weight, wingspan)
# oy: target value (1)
oX, oy = species_generator(owl_weight_mean, owl_weight_variance,
                           owl_wingspan_mean, owl_wingspan_variance,
                           n_samples,target,seed )

owl_dic = {'weight-(gm)': oX[:,0],
             'wingspan-(cm)': oX[:,1], 
             'species': oy,
             'url': "https://raw.githubusercontent.com/pabloinsente/nn-mod-cog/master/notebooks/images/owl.png"}

# put values in a relational table (pandas dataframe)
owl_df = pd.DataFrame(owl_dic)

In [180]:
# df = albatross_df.append(owl_df, ignore_index=True)
df = pd.concat([albatross_df, owl_df], ignore_index=True)

alt.Chart(df).mark_image(
    width=20,
    height=20
).encode(
    x="weight-(gm)",
    y="wingspan-(cm)",
    url="url"
).properties(
    title='Chart 1'
)

alt.Chart(...)

In [181]:
df_shuffle = df.sample(frac=1, random_state=1).reset_index(drop=True)
X = df_shuffle[['weight-(gm)','wingspan-(cm)']].to_numpy()
y = df_shuffle['species'].to_numpy()

In [182]:
w, errors = fit(X, y, eta=0.01, n_iter=200)
print(w, errors)

[  -39.42375655    96.03429975 -1943.76432849] [69, 87, 79, 7, 19, 19, 46, 19, 19, 19, 36, 19, 36, 47, 19, 46, 19, 19, 36, 27, 9, 27, 26, 9, 26, 9, 26, 9, 26, 9, 26, 9, 26, 9, 9, 18, 9, 9, 26, 9, 26, 9, 26, 9, 26, 9, 18, 8, 8, 17, 9, 17, 8, 18, 17, 9, 17, 17, 8, 17, 9, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 9, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 15, 16, 15, 16, 8, 8, 8, 8, 8, 1, 16, 15, 8, 8, 8, 8, 8, 7, 1, 15, 8, 8, 8, 7, 1, 14, 1, 14, 1, 14, 1, 15, 8, 8, 8, 7, 1, 14, 1, 14, 1, 14, 1, 14, 1, 8, 15, 1, 14, 1, 14, 1, 14, 1, 14, 1, 8, 15, 1, 14, 1, 14, 1, 14, 1, 14, 1, 8, 8, 8, 8, 8, 8, 7, 1, 14, 1, 14, 1]


In [183]:
y_pred = predict(X, w)
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Perceptron accuracy: %.2f%%' % accuracy)

Perceptron accuracy: 99.50%


In [184]:
error_df = pd.DataFrame({'error':errors, 'time-step': np.arange(0, len(errors))})

alt.Chart(error_df).mark_line().encode(
    x="time-step", y="error"
).properties(
    title='Chart 2'
)

alt.Chart(...)

In [185]:
condor_weight_mean = 12000 # in grams
condor_weight_variance = 1000 # in grams
condor_wingspan_mean = 290 # in cm
condor_wingspan_variance = 15 # in cm 
n_samples = 100
target = -1
seed = 100

# cX: feature matrix (weight, wingspan)
# cy: target value (1)
cX, cy = species_generator(condor_weight_mean, condor_weight_variance,
                           condor_wingspan_mean, condor_wingspan_variance,
                           n_samples,target,seed )

condor_dic = {'weight-(gm)': cX[:,0],
             'wingspan-(cm)': cX[:,1], 
             'species': cy,
             'url': "https://raw.githubusercontent.com/pabloinsente/nn-mod-cog/master/notebooks/images/condor.png"}

# put values in a relational table (pandas dataframe)
condor_df = pd.DataFrame(condor_dic)

df2 = pd.concat([albatross_df, condor_df], ignore_index=True)

In [186]:
alt.Chart(df2).mark_image(
    width=20,
    height=20
).encode(
    alt.X("weight-(gm)", scale=alt.Scale(domain=(6000, 16000))),
    alt.Y("wingspan-(cm)", scale=alt.Scale(domain=(220, 360))),
    url="url"
).properties(
title="Chart 3"
)

alt.Chart(...)

In [187]:
df_shuffle2 = df2.sample(frac=1, random_state=1).reset_index(drop=True)
X = df_shuffle2[['weight-(gm)','wingspan-(cm)']].to_numpy()
y = df_shuffle2['species'].to_numpy()

In [192]:
w, errors = fit(X, y, eta=0.01, n_iter=1000)

In [194]:
y_pred = predict(X, w)
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Perceptron accuracy: %.2f%%' % accuracy)

Perceptron accuracy: 94.00%


In [195]:
error_df = pd.DataFrame({'error':errors, 'time-step': np.arange(0, len(errors))})
alt.Chart(error_df).mark_line().encode(
    x="time-step", y="error"
).properties(
    title='Chart 4'
)

alt.Chart(...)